In [1]:
import json
import numpy as np
import os
import lib.helper as hl

In [2]:
current_dir = os.getcwd()
dir_general = os.path.join(current_dir,'data/results/')
sim_used = '2018_12_5_14_1_all'

In [3]:
dir_cell_results = os.path.join(dir_general,sim_used,'cell/results/')
dir_cell_params = os.path.join(dir_general,sim_used,'cell/params/')

In [4]:
'''
search for all the results obtained by simulating single cells and sums them toghether
it adds /cell/results to dir_general to find the dir with the results
and it saves it in /all_results/
'''

cell_files = hl.find_files(dir_cell_results, ext='npz')  # get all the cell results

inh_syn_x, inh_syn_y, inh_syn_z = [], [], []
exc_syn_x, exc_syn_y, exc_syn_z = [], [], []

# load all params
with file(os.path.join(dir_general, sim_used)+'.json', 'r') as fid:
    params_all = json.load(fid)
y_range = params_all['record_y_range']
x_range = params_all['record_x_range']

In [5]:
def add_syn_coords_to_list(syn_x, syn_y, syn_z, list_of_coords):
    if len(list_of_coords) == 0:
        return syn_x, syn_y, syn_z
    else:
        for next_coord in list_of_coords:
            syn_x.append(next_coord[0])
            syn_y.append(next_coord[1])
            syn_z.append(next_coord[2])
        return syn_x, syn_y, syn_z

In [6]:
idx_mf = 0
for idx, cell_file in enumerate(cell_files):
    print 'summing cell: ', idx, '/', len(cell_files), ' : ',  cell_file[:-4], '.json'
    results = np.load(os.path.join(dir_cell_results, cell_file))
    
    with file(os.path.join(dir_cell_params, cell_file[:-4])+'.json', 'r') as fid:
        params = json.load(fid)
        
    # only if the field was calculated add it to the sum
    if ('v_ext' in results.keys()):
        if idx == 0:
            v_ext = results['v_ext']
        else:
            v_ext += results['v_ext']
    inh_syn_x, inh_syn_y, inh_syn_z=add_syn_coords_to_list(inh_syn_x, inh_syn_y, inh_syn_z, params['inh_syn_coords'])
    exc_syn_x, exc_syn_y, exc_syn_z=add_syn_coords_to_list(exc_syn_x, exc_syn_y, exc_syn_z, params['exc_syn_coords'])
inh_syn_coords = zip(inh_syn_x, inh_syn_y, inh_syn_z)
exc_syn_coords = zip(exc_syn_x, exc_syn_y, exc_syn_z)

summing cell:  0 / 10  :  c10861.CNG9 .json
summing cell:  1 / 10  :  c10861.CNG7 .json
summing cell:  2 / 10  :  c10861.CNG6 .json
summing cell:  3 / 10  :  c10861.CNG2 .json
summing cell:  4 / 10  :  c10861.CNG1 .json
summing cell:  5 / 10  :  c10861.CNG5 .json
summing cell:  6 / 10  :  c10861.CNG4 .json
summing cell:  7 / 10  :  c10861.CNG3 .json
summing cell:  8 / 10  :  c10861.CNG8 .json
summing cell:  9 / 10  :  c10861.CNG0 .json


In [7]:
# save the summed field
savefile = os.path.join(dir_general, sim_used, 'results_all.npz')
if ('v_ext' in results.keys()):
    np.savez(savefile, v_ext=v_ext,
             inh_syn_coords=inh_syn_coords,exc_syn_coords=exc_syn_coords,
             record_x_range=x_range, record_y_range=y_range)
else:
    np.savez(savefile,
                 inh_syn_coords=inh_syn_coords, exc_syn_coords=exc_syn_coords)
print 'results were saved in ', savefile

results were saved in  /home/maria/maja/code/lfp-paper/data/results/2018_12_5_14_1_all/results_all.npz
